In [1]:
import pandas as pd 
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

### 데이터 불러오기 

In [2]:
cust_tr = pd.read_csv('cust_train.csv', encoding = 'utf-8') # 회원
product_tr = pd.read_csv('product_train.csv',encoding = 'utf-8') # 상품구매
search_tr = pd.read_csv('search_train.csv',encoding = 'utf-8') # 검색어
session_tr = pd.read_csv('session_train.csv',encoding = 'utf-8') # 세션

In [3]:
cust_te = pd.read_csv('cust_test.csv', encoding = 'utf-8')
product_te = pd.read_csv('product_test.csv',encoding = 'utf-8')
search_te = pd.read_csv('search_test.csv',encoding = 'utf-8')
session_te = pd.read_csv('session_test.csv',encoding = 'utf-8')

In [4]:
master = pd.read_csv('master.csv', encoding = 'utf-8') # 상품분류

### 데이터 전처리

In [5]:
train = pd.merge(product_tr, session_tr, on = ['CLNT_ID','SESS_ID'])
train = pd.merge(train, search_tr, on = ['CLNT_ID','SESS_ID'])
train = pd.merge(train, master, on = 'PD_C')

In [6]:
test = pd.merge(product_te, session_te, on = ['CLNT_ID','SESS_ID'])
test = pd.merge(test, search_te, on = ['CLNT_ID','SESS_ID'])
test = pd.merge(test, master, on = 'PD_C')

In [7]:
train['PD_BUY_AM'] = train['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
train['PD_BUY_CT'] = train['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_AM'] = test['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_CT'] = test['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )

- 상품 1개당 금액과 구매수량을 int 타입으로 바꿈

### feature 생성

In [107]:
features = []
features_te = []

**[총구매액, 구매건수, 평균구매액, 최대구매액]**

In [108]:
train['AMOUNT'] = train['PD_BUY_AM'] * train['PD_BUY_CT']
test['AMOUNT'] = test['PD_BUY_AM'] * test['PD_BUY_CT']

- 한 사람이 구매한 총 금액을  알기 위해 추가

In [109]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features_te.append(f_te) ; display(f_te)

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,0,73400,4,18350,32000
1,1,84000,3,28000,28000
2,2,106400,12,8867,9900
3,3,202500,9,22500,39200
4,4,1116500,32,34891,65000
...,...,...,...,...,...
263099,263099,819870,3,273290,744870
263100,263100,501500,10,50150,74000
263101,263101,64760,2,32380,33000
263102,263102,93600,3,31200,31200


,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,263104,31900,1,31900,31900
1,263105,33000,1,33000,33000
2,263106,116000,4,29000,29000
3,263107,70000,2,35000,35000
4,263108,487500,5,97500,129000
...,...,...,...,...,...
112755,375859,52200,2,26100,26100
112756,375860,624000,21,29714,69000
112757,375861,569920,17,33525,325000
112758,375862,614600,11,55873,90000


**[주말방문비율]**

In [110]:
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')

In [111]:
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

- 구매일자를 datetime 타입으로 변형

In [112]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features_te.append(f_te);display(f_te)

,CLNT_ID,주말방문비율
0,0,0.000000
1,1,0.666667
2,2,1.000000
3,3,1.000000
4,4,0.062500
...,...,...
263099,263099,0.000000
263100,263100,0.600000
263101,263101,1.000000
263102,263102,1.000000


,CLNT_ID,주말방문비율
0,263104,1.000000
1,263105,0.000000
2,263106,0.000000
3,263107,0.000000
4,263108,0.000000
...,...,...
112755,375859,0.000000
112756,375860,0.190476
112757,375861,0.000000
112758,375862,0.272727


**[계절방문비율]**

In [113]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f); f

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,0,0.000000,1.000000,0.000000,0.0
1,1,0.000000,0.333333,0.666667,0.0
2,2,0.000000,0.000000,1.000000,0.0
3,3,0.000000,0.000000,1.000000,0.0
4,4,0.625000,0.375000,0.000000,0.0
...,...,...,...,...,...
263099,263099,0.666667,0.333333,0.000000,0.0
263100,263100,0.300000,0.700000,0.000000,0.0
263101,263101,1.000000,0.000000,0.000000,0.0
263102,263102,1.000000,0.000000,0.000000,0.0


In [114]:
f_te = test.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12])))
]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,263104,1.000000,0.000000,0.0,0.0
1,263105,0.000000,0.000000,1.0,0.0
2,263106,1.000000,0.000000,0.0,0.0
3,263107,1.000000,0.000000,0.0,0.0
4,263108,0.800000,0.200000,0.0,0.0
...,...,...,...,...,...
112755,375859,0.000000,1.000000,0.0,0.0
112756,375860,0.666667,0.333333,0.0,0.0
112757,375861,0.647059,0.352941,0.0,0.0
112758,375862,0.636364,0.363636,0.0,0.0


### Merge features

In [68]:
data = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})
for f in features :
    data = pd.merge(data, f, how='left')
    
data = data.fillna(0)

In [69]:
data_te = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})
for f in features_te :
    data_te = pd.merge(data_te, f, how='left')
    
data_te = data_te.fillna(0)

### 새로운 numeric 변수 추가하기

**[주차별방문비율]**

In [120]:
train['day'] = train['date'].dt.strftime('%d')
train['day'] = train['day'].astype('int')

train['month_group'] = pd.cut(train['day'], bins = [1, 10, 20, 31], 
                              right = False, labels = ['월초', '월중', '월말'])

test['day'] = test['date'].dt.strftime('%d')
test['day'] = test['day'].astype('int')

test['month_group'] = pd.cut(test['day'], bins = [1, 10, 20, 31], 
                              right = False, labels = ['월초', '월중', '월말'])

In [121]:
f = train.groupby('CLNT_ID')['month_group'].agg([('월3등분', 'value_counts')]).reset_index()
f = pd.pivot_table(f, index = 'CLNT_ID', columns = 'month_group', values = '월3등분', fill_value = 0).reset_index()
f = f.reindex(columns = ['CLNT_ID', '월초', '월중', '월말'])
features.append(f);f

month_group,CLNT_ID,월초,월중,월말
0,0,0,0,4
1,1,2,0,1
2,2,12,0,0
3,3,0,0,9
4,4,6,18,8
...,...,...,...,...
261244,263099,0,3,0
261245,263100,2,3,5
261246,263101,0,0,2
261247,263102,0,0,3


In [122]:
f_te = test.groupby('CLNT_ID')['month_group'].agg([('월3등분', 'value_counts')]).reset_index()
f_te = pd.pivot_table(f_te, index = 'CLNT_ID', columns = 'month_group', values = '월3등분', fill_value = 0).reset_index()
f_te = f_te.reindex(columns = ['CLNT_ID', '월초', '월중', '월말'])
features.append(f);f

month_group,CLNT_ID,월초,월중,월말
0,0,0,0,4
1,1,2,0,1
2,2,12,0,0
3,3,0,0,9
4,4,6,18,8
...,...,...,...,...
261244,263099,0,3,0
261245,263100,2,3,5
261246,263101,0,0,2
261247,263102,0,0,3


**[페이지 당 머무른 평균 시간]**

In [126]:
# object 타입인 TOT_SESS_HR_V의 int로 바꿔줌
# train['TOT_SESS_HR_V'] = train['TOT_SESS_HR_V'].str.replace(',', '').astype('int64')
t# est['TOT_SESS_HR_V'] = test['TOT_SESS_HR_V'].str.replace(',', '').astype('int64')

NameError: name 't' is not defined

In [127]:
train['page_aver'] = train['TOT_SESS_HR_V'] / train['TOT_PAG_VIEW_CT']
test['page_aver'] = test['TOT_SESS_HR_V'] / test['TOT_PAG_VIEW_CT']

In [128]:
f = train.groupby('CLNT_ID')['page_aver'].agg([('페이지 당 머무른 평균 시간', np.mean)]).reset_index()
features.append(f); display(f)

,CLNT_ID,페이지 당 머무른 평균 시간
0,0,6.730769
1,1,10.952614
2,2,15.151042
3,3,34.585366
4,4,16.679401
...,...,...
263099,263099,13.890259
263100,263100,17.228470
263101,263101,17.246377
263102,263102,10.916667


In [129]:
f_te = test.groupby('CLNT_ID')['page_aver'].agg([('페이지 당 머무른 평균 시간', np.mean)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,페이지 당 머무른 평균 시간
0,263104,13.256410
1,263105,16.032967
2,263106,18.869159
3,263107,9.194444
4,263108,20.774263
...,...,...
112755,375859,15.141361
112756,375860,14.641162
112757,375861,25.838310
112758,375862,11.369300


**[총 접속시간]**

In [130]:
f = train.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([('총 접속시간', sum)]).reset_index()
features.append(f); display(f)

,CLNT_ID,총 접속시간
0,0,4200
1,1,2805
2,2,34908
3,3,63810
4,4,77542
...,...,...
263099,263099,6029
263100,263100,19822
263101,263101,2380
263102,263102,4716


In [131]:
f_te = test.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([('총 접속시간', sum)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,총 접속시간
0,263104,517
1,263105,1459
2,263106,16152
3,263107,662
4,263108,17009
...,...,...
112755,375859,5784
112756,375860,46129
112757,375861,77664
112758,375862,38135


**[접속한 세션 수]**

In [132]:
f = train.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].agg([('접속한 세션 수', sum)]).reset_index()
features.append(f); display(f)

,CLNT_ID,접속한 세션 수
0,0,624.0
1,1,272.0
2,2,2304.0
3,3,1845.0
4,4,6070.0
...,...,...
263099,263099,419.0
263100,263100,1735.0
263101,263101,138.0
263102,263102,432.0


In [133]:
f_te = test.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].agg([('접속한 세션 수', sum)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,접속한 세션 수
0,263104,39.0
1,263105,91.0
2,263106,856.0
3,263107,72.0
4,263108,783.0
...,...,...
112755,375859,382.0
112756,375860,3182.0
112757,375861,3105.0
112758,375862,3428.0


**[총 검색건수]**

In [134]:
f = train.groupby('CLNT_ID')['SEARCH_CNT'].agg([('총 검색건수', sum)]).reset_index()
features.append(f); display(f)

,CLNT_ID,총 검색건수
0,0,4
1,1,8
2,2,15
3,3,12
4,4,38
...,...,...
263099,263099,7
263100,263100,14
263101,263101,6
263102,263102,5


In [135]:
f_te = test.groupby('CLNT_ID')['SEARCH_CNT'].agg([('총 검색건수', sum)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,총 검색건수
0,263104,1
1,263105,1
2,263106,4
3,263107,2
4,263108,5
...,...,...
112755,375859,2
112756,375860,25
112757,375861,54
112758,375862,17


**[평균 검색건수]**

In [136]:
f = train.groupby('CLNT_ID')['SEARCH_CNT'].agg([('평균 검색건수', np.mean)]).reset_index()
features.append(f); display(f)

,CLNT_ID,평균 검색건수
0,0,1.000000
1,1,2.666667
2,2,1.250000
3,3,1.333333
4,4,1.187500
...,...,...
263099,263099,2.333333
263100,263100,1.400000
263101,263101,3.000000
263102,263102,1.666667


In [137]:
f_te = test.groupby('CLNT_ID')['SEARCH_CNT'].agg([('평균 검색건수', np.mean)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,평균 검색건수
0,263104,1.000000
1,263105,1.000000
2,263106,1.000000
3,263107,1.000000
4,263108,1.000000
...,...,...
112755,375859,1.000000
112756,375860,1.190476
112757,375861,3.176471
112758,375862,1.545455


**[주 사용기기]**

In [138]:
def y(device):
    if device == 'desktop': x=1
    elif device == 'mobile': x=2
    else : x=3
    return x

train['DVC_CTG_NM'] = train['DVC_CTG_NM'].apply(y)
test['DVC_CTG_NM'] = test['DVC_CTG_NM'].apply(y)
test['DVC_CTG_NM']

0          3
1          3
2          3
3          3
4          3
          ..
1461072    3
1461073    3
1461074    3
1461075    3
1461076    3
Name: DVC_CTG_NM, Length: 1461077, dtype: int64

In [139]:
def g(x):
    y = x.value_counts().index[0]
    return y

f = train.groupby('CLNT_ID')['DVC_CTG_NM'].agg([('주 사용기기', g)]).reset_index()
features.append(f); display(f)

,CLNT_ID,주 사용기기
0,0,3
1,1,3
2,2,3
3,3,3
4,4,3
...,...,...
263099,263099,3
263100,263100,3
263101,263101,3
263102,263102,3


In [140]:
f_te = test.groupby('CLNT_ID')['DVC_CTG_NM'].agg([('주 사용기기', g)]).reset_index()
features.append(f_te); display(f_te)

,CLNT_ID,주 사용기기
0,263104,3
1,263105,3
2,263106,3
3,263107,3
4,263108,3
...,...,...
112755,375859,3
112756,375860,3
112757,375861,3
112758,375862,3


**[사용 브랜드 개수]**

In [141]:
f = train.groupby('CLNT_ID')['PD_BRA_NM'].agg([('사용 브랜드 개수', 'nunique')]).reset_index()
features.append(f); display(f)

,CLNT_ID,사용 브랜드 개수
0,0,2
1,1,1
2,2,3
3,3,3
4,4,8
...,...,...
263099,263099,2
263100,263100,4
263101,263101,1
263102,263102,1


In [142]:
f_te = test.groupby('CLNT_ID')['PD_BRA_NM'].agg([('사용 브랜드 개수', 'nunique')]).reset_index()
features.append(f_te); display(f_te)

,CLNT_ID,사용 브랜드 개수
0,263104,1
1,263105,1
2,263106,1
3,263107,1
4,263108,2
...,...,...
112755,375859,1
112756,375860,4
112757,375861,5
112758,375862,2


**[최소 구매액]**

In [143]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('최소 구매액', 'min')]).reset_index()
features.append(f); display(f)

,CLNT_ID,최소 구매액
0,0,4700
1,1,28000
2,2,7900
3,3,12000
4,4,9000
...,...,...
263099,263099,37500
263100,263100,39500
263101,263101,31760
263102,263102,31200


In [144]:
f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('최소 구매액', 'min')]).reset_index()
features.append(f_te); display(f_te)

,CLNT_ID,최소 구매액
0,263104,31900
1,263105,33000
2,263106,29000
3,263107,35000
4,263108,31500
...,...,...
112755,375859,26100
112756,375860,6500
112757,375861,3790
112758,375862,27800


**[단일상품 최대금액]**

In [145]:
f = train.groupby('CLNT_ID')['PD_BUY_AM'].agg([('단일상품 최대금액', 'max')]).reset_index()
features.append(f); display(f)

,CLNT_ID,단일상품 최대금액
0,0,32000
1,1,28000
2,2,9900
3,3,39200
4,4,65000
...,...,...
263099,263099,744870
263100,263100,74000
263101,263101,33000
263102,263102,31200


In [146]:
f_te = test.groupby('CLNT_ID')['PD_BUY_AM'].agg([('단일상품 최대금액', 'max')]).reset_index()
features.append(f_te); display(f_te)

,CLNT_ID,단일상품 최대금액
0,263104,31900
1,263105,33000
2,263106,29000
3,263107,35000
4,263108,129000
...,...,...
112755,375859,26100
112756,375860,69000
112757,375861,325000
112758,375862,90000


In [147]:
data = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})
for f in features :
    data = pd.merge(data, f, how='left')
    
data = data.fillna(0)

In [148]:
data_te = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})
for f in features_te :
    data_te = pd.merge(data_te, f, how='left')
    
data_te = data_te.fillna(0)

In [149]:
data

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액,주말방문비율,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율,...,월말,페이지 당 머무른 평균 시간,총 접속시간,접속한 세션 수,총 검색건수,평균 검색건수,주 사용기기,사용 브랜드 개수,최소 구매액,단일상품 최대금액
0,121165,1119180,22,50872,143200,0.454545,0.227273,0.772727,0.0,0.0,...,5.0,18.280516,63475,3839.0,27,1.227273,3,12,9600,143200
1,130759,730500,89,8208,52200,0.224719,0.000000,1.000000,0.0,0.0,...,12.0,17.280524,279793,17459.0,103,1.157303,3,14,100,52200
2,43665,2031000,86,23616,53100,0.046512,0.046512,0.953488,0.0,0.0,...,3.0,7.171881,270896,40283.0,207,2.406977,3,7,2900,53100
3,27258,201360,7,28766,39000,0.142857,0.000000,1.000000,0.0,0.0,...,1.0,13.351097,12589,938.0,7,1.000000,3,2,20800,39000
4,137634,482400,12,40200,49800,0.000000,0.000000,0.000000,1.0,0.0,...,0.0,16.516484,18036,1092.0,16,1.333333,3,1,32800,49800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263099,122358,39900,1,39900,39900,0.000000,1.000000,0.000000,0.0,0.0,...,0.0,10.304348,474,46.0,1,1.000000,3,1,39900,39900
263100,49419,39000,1,39000,39000,0.000000,0.000000,0.000000,1.0,0.0,...,0.0,26.020833,2498,96.0,2,2.000000,3,1,39000,39000
263101,175011,39200,1,39200,39200,1.000000,0.000000,1.000000,0.0,0.0,...,0.0,18.379085,2812,153.0,2,2.000000,3,1,39200,39200
263102,237452,39000,1,39000,39000,1.000000,0.000000,1.000000,0.0,0.0,...,0.0,8.615385,896,104.0,4,4.000000,3,1,39000,39000
